#**Krabbe Valentina- 45207992**

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [8]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [9]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [10]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [11]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [12]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [13]:
cossim

array([0.1382319 , 0.1067036 , 0.23029327, ..., 0.12320753, 0.08765353,
       0.04415046])

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 2

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.



In [22]:
#Cargamos los datos (ya separados de forma predeterminada en train y test)
news_groups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
news_groups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [23]:
#Instanciamos un vectorizador
tfidfvect=TfidfVectorizer()

In [24]:
#Convertimos los documentos de texto en vectores numericos usando TF-IDF, e indica la importancia de una palabra en un documento
X_train=tfidfvect.fit_transform(news_groups_train.data)
#Podemos denominar a X_train como la matriz documento-termino.

In [25]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas

print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [26]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [27]:
# en `y_train` guardamos los targets que son enteros
y_train = news_groups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

##Similaridad de documentos

In [33]:
np.random.seed(47)
#Seleccionamos 5 documentos aleatorios
random_docs=np.random.choice(X_train.shape[0],size=5, replace=False)

In [34]:
for idx in random_docs:
  clase=news_groups_train.target[idx]
  nombre_clase=news_groups_train.target_names[clase]
  print('='*80)
  print(f'\n Documento base {idx} - Clase:{clase}({nombre_clase})')
  print(news_groups_train.data[idx][:300],'...\n')
  print('='*80)


 Documento base 5073 - Clase:10(rec.sport.hockey)
Hi.  According to my always questionable newspaper, there's a silver
dollar being minted in Canada, the back of which commemorates the
anniversary of the Stanley Cup.  Now, I know full well what coins are
used every day in Canada.  I can easily fish a few out of my pocket
change right now, in fact.  ...


 Documento base 4819 - Clase:13(sci.med)


Krillean photography involves taking pictures of minute decapods resident in 
the seas surrounding the antarctic. Or pictures taken by them, perhaps. ...


 Documento base 10919 - Clase:12(sci.electronics)


Take a lemon.  Stick a copper strip into the lemon.  Stick a galvanized
nail in about 1 to 1-1/2 inches from the copper strip.  You should get
about 1/2 volt from it - enough to light an LED.


 * SLMR 2.1 * Support your medical examiner, die strangely.
                      ...


 Documento base 5549 - Clase:0(alt.atheism)

You're closer than you might imagine.   I certainly despised liv

In [35]:
for idx in random_docs:
  #Medimos la similaridad coseno con todos los documentos de train
  cossim=cosine_similarity(X_train[idx], X_train)[0]

  #5 documentos mas similares:
  mostsim=np.argsort(cossim)[::-1][1:6]

  for i, sim_idx in enumerate(mostsim): #sim_idx almacena cada resultado del bucle
    print('='*80)
    print(f'Similar: {i+1} (indice {sim_idx}, similitud {round(cossim[sim_idx],2)})')
    print(f'-Categoria: {news_groups_train.target_names[y_train[sim_idx]]}')
    print(f'-Preview: {news_groups_train.data[sim_idx][:250]}...\n')


Similar: 1 (indice 1017, similitud 0.26)
-Categoria: rec.autos
-Preview: Sayeth sjwyrick@lbl.gov (Steve Wyrick):
$Anybody keeping track of how many of these there are?  So far I have
$Miata, Tredia, Previa, Sentra, Maxima, Altima, Camry, and Justy, not to
$mention Lexus, Acura and Infiniti!  

   You're apparently includi...

Similar: 2 (indice 3568, similitud 0.25)
-Categoria: talk.religion.misc
-Preview: In <lsjc8cINNmc1@saltillo.cs.utexas.edu> turpin@cs.utexas.edu (Russell Turpin)


I regard love as no more or less "benign" than any other Christian does.
You are merely expressing "approval" of the consequences I find therein.
Which says more about o...

Similar: 3 (indice 10836, similitud 0.25)
-Categoria: alt.atheism
-Preview: Archive-name: atheism/faq
Alt-atheism-archive-name: faq
Last-modified: 5 April 1993
Version: 1.1

                    Alt.Atheism Frequently-Asked Questions

This file contains responses to articles which occur repeatedly in
alt.atheism.  Points cove...

Simi

En cocnlusion seleccionamos 5 documentos al azar del conjunto '20 Newsgroups' y para cada uno se calculo los 5 documentos mas similares.

El analisis de similitud muestra que la mayoria de los documentos mas parecidos provienen de categorias imilares, como 'talk.politics.mideast', lo que indica que estan relacionados en tema. Por otro lado lo documentos con meno similitud son de categorias diferentes, como 'misc.forsale' o 'sci.crypt', lo que demuestra que aunque. Esto resalta la importancia de tener en cuenta el contexto y la categoria de los documentos al analizar su similitud.


**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [36]:
#Vectorizamos el texto con parametros modificados
tfidf_vect=TfidfVectorizer(ngram_range=(1,2),max_features=10000)
X_train=tfidf_vect.fit_transform(news_groups_train.data)
X_test=tfidf_vect.transform(news_groups_test.data)

#Definimos las etiquetas
y_train=news_groups_train.target
y_test=news_groups_test.target

In [38]:
#Defino valores de alpha a probar
alphas=np.round(np.linspace(0.05,0.25,5),3)

#Iteramos sobre los valores alpha:
for alpha in alphas:
  #Instanciamos y entrenamos los dos modelos con el mismo alpha
  clf_Mult=MultinomialNB(alpha=alpha)
  clf_compl=ComplementNB(alpha=alpha)

  clf_Mult.fit(X_train,y_train)
  clf_compl.fit(X_train,y_train)

  #Hacemos las predicciones sobre el conjunto del test
  y_pred_Mult=clf_Mult.predict(X_test)
  y_pred_compl=clf_compl.predict(X_test)

  #Calculamos el F1-Score macro para ambos modelos
  f1_m=f1_score(y_test,y_pred_Mult,average='macro')
  f1_c=f1_score(y_test,y_pred_compl,average='macro')

  #Imprimimos los resultados
  print(f'Alpha={alpha:.2f}| MultinomialNB: {f1_m:.4f} | ComplementNB: {f1_c:.4f}')

Alpha=0.05| MultinomialNB: 0.6166 | ComplementNB: 0.6294
Alpha=0.10| MultinomialNB: 0.6188 | ComplementNB: 0.6306
Alpha=0.15| MultinomialNB: 0.6174 | ComplementNB: 0.6309
Alpha=0.20| MultinomialNB: 0.6166 | ComplementNB: 0.6314
Alpha=0.25| MultinomialNB: 0.6119 | ComplementNB: 0.6313


Podemos notar que ComplementNB tiene mejor rendimiento que MultinomialNB en todas los valores de alpha que se probaron.

Probe los dos modelos con distintos valores de alpha para ver cual clasificaba mejor, usando F1-Score macro como metrica porque tieneen cuenta todas las clases por igual.
En todos los casos, el modelo ComplementNB dio mejores resultados, y el mejorfue alpha=0.20, que llego a un f1-score=0.6314. Mostrando que ese modelo funciona mejor con ese tipo de datos



**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [40]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#categorías seleccionadas del dataset
categories = [
    'sci.med', #medicina
    'rec.autos', #autos
    'comp.sys.mac.hardware', #hardware de Mac
    'alt.atheism', #ateismo
    'sci.crypt' #criptografia
]

#cargamos corpus de textos (limitamos a 100 documentos para velocidad)
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
corpus = newsgroups.data[:100]

#vectorizamos el texto del corpus
vectorizer = CountVectorizer(stop_words='english') #ignora palabras comnes como 'the', 'and', etc
X = vectorizer.fit_transform(corpus)#aplica la vectorizacion al corpus
dtm = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

#trnasponemos: DTM para tener la TDM, donde las filas son palabras
tdm = dtm.T

#palabras seleccionadas
palabras_interes = ["medicine", "car", "mac", "drug", "encryption"]
#filtro para ver si als palabras si estan en la TDM
palabras_validas = [p for p in palabras_interes if p in tdm.index]

#calculamos similitud de coseno entre las palabras seleccionadas y las demas palabras
similaridades = cosine_similarity(tdm.loc[palabras_validas], tdm)
#creamo un df con los resultados
similaridades_df = pd.DataFrame(similaridades, index=palabras_validas, columns=tdm.index)

#mostramos las 5 palabras más similares para cada palabra clave
for palabra in palabras_validas:
    print(f"\nPalabra clave: {palabra}")
    similares = similaridades_df.loc[palabra].sort_values(ascending=False)[1:6]
    print(similares)


Palabra clave: medicine
surpress      0.96225
absolutely    0.96225
aritcle       0.96225
deeply        0.96225
swiss         0.96225
Name: medicine, dtype: float64

Palabra clave: car
actually    0.969363
owned       0.898478
calling     0.896922
careful     0.896922
100k        0.896922
Name: car, dtype: float64

Palabra clave: mac
mac         1.000000
software    0.750000
apple       0.731925
cabling     0.707107
2775        0.707107
Name: mac, dtype: float64

Palabra clave: drug
dealers       0.816497
protection    0.816497
encryption    0.800641
plenty        0.707107
annul         0.707107
Name: drug, dtype: float64

Palabra clave: encryption
wants           0.880705
cryptography    0.818587
length          0.810191
drug            0.800641
secret          0.708918
Name: encryption, dtype: float64


Podemos ver lo siguiente:

> **medicine**: Aparecen palabras como [surpess, deeply, swiss] que no son terminos medicos directos, pero se nota que son parte del mismo contexto en el que se habla en medicina.

> **car**: Se asocian las palabras [owned, 100k, careful] que claramente estan relacionadas

> **mac**: Tambien presenta palabras profundamente relacionadas como [apple, software, cablin], lo caul muestra que el modelo entendio bien el contexto tecnologico.

> **drug**: Aparecen palabras como [dealers,protection y encryption]

> **encryption**: Tiene una relacion fuerte con [crtiptography, secret, lenght], palabras fuertemente relacionadas con el cifrado de datos. Ademas se conecta con [drug] lo que sugiere que en textos esos temas aparecen juntos, por debates sobre privacidad, seguridad, etc.


###En conclusion podemos decir que la **similitud de coseno** funciona bien para encontrar palabras que comparten significado.